This Program is a combination of the first three prompts in assignment #1.

Part one:
 Write a program to learn the best polynomial model using the least square approach by only increasing or decreasing 
 the polynomial order (without using any method to improve the accuracy of the model)

Part two: 
Write a program without using any library or package to learn the best polynomial model using the gradient descent method 
discussed in class by only increasing or decreasing the polynomial order (NUMPY AND PANDAS NOT ALLOWED)

Part three:
Write a program for a feature scaling and run one of the programs written in either #1/#2 to learn the best
polynomial model with the scaled data set 

Resources:
1. https://linguisticmaz.medium.com/implementing-polynomial-regression-in-python-d9aedf520d56
2. https://realpython.com/visualizing-python-plt-scatter/
3. https://satishgunjal.com/multivariate_lr/
4. https://medium.com/@lope.ai/multivariate-linear-regression-from-scratch-in-python-5c4f219be6a
5. https://integratedmlai.com/gradient-descent-using-pure-python-without-numpy-or-scipy/
6. https://stackoverflow.com/questions/43697240/how-can-i-split-a-dataset-from-a-csv-file-for-training-and-testing
7. https://data-flair.training/blogs/train-test-set-in-python-ml/
8. https://www.askpython.com/python/coefficient-of-determination
9. https://www.statology.org/r-squared-in-python/
10. https://pythonexamples.org/python-list-with-first-n-elements/
11. https://towardsdatascience.com/data-visualization-for-machine-learning-and-data-science-a45178970be7
12. https://stackoverflow.com/questions/47202182/train-test-split-without-using-scikit-learn


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
#loading data 
gas_dataset=pd.read_csv('/Users/maddie/Projects/CPSC_483/assignmnet_1/Data1.csv')

#creating function to  randomly split the data into training, validation and testing set
def split_train_valid_test(data,valid_ratio,test_ratio):
    shuffled_indcies=np.random.permutation(len(data))
    valid_set_size= int(len(data)*valid_ratio)
    valid_indcies=shuffled_indcies[:valid_set_size]
    test_set_size= int(len(data)*test_ratio)
    test_indcies=shuffled_indcies[valid_set_size:test_set_size+valid_set_size]
    train_indices=shuffled_indcies[test_set_size:]
    return data.iloc[train_indices],data.iloc[valid_indcies],data.iloc[test_indcies]

#setting ratio of data
train_set,valid_set,test_set=split_train_valid_test(gas_dataset,valid_ratio=0,test_ratio=0.2)

#splitting  training data into x and y variables, displaying value of training variables
train_x = train_set.values[:, 0:4]  
train_y = train_set.values[:, 4]  
test_x = test_set.values[:, 0:4]  
test_y = test_set.values[:, 4]
m = len(train_y) 
n = len(test_y)
print('Total no of training examples (m) = %s \n' %(m), 'Total number of test values (n) = %s \n' %(n))

#creating a feature normalization function to help with gradient descent
def feature_normalize(train_x):
  mu = np.mean(train_x, axis = 0)  
  sigma = np.std(train_x, axis= 0, ddof = 1) 
  x_norm = (train_x - mu)/sigma
  return x_norm, mu, sigma

#calling feature normalization function, displaying first five results 
train_x, mu, sigma = feature_normalize(train_x)

#finding our new normalized means, std. dev, and adding a column of ones to our training set
mu_testing = np.mean(train_x, axis = 0) # mean
mu_testing
sigma_testing = np.std(train_x, axis = 0, ddof = 1) # mean
sigma_testing
train_x = np.hstack((np.ones((m,1)), train_x))
train_x[:5]

#creating the multivariate regression function
def compute_cost(train_x, train_y, theta):
  predictions = train_x.dot(theta)
  errors = np.subtract(predictions, train_y)
  sqrErrors = np.square(errors)
  J = 1/(2 * m) * errors.T.dot(errors)
  return J

#creating the gradient descent function
def gradient_descent(train_x, train_y, theta, alpha, iterations):
  cost_history = np.zeros(iterations)
  for i in range(iterations):
    predictions = train_x.dot(theta)   
    errors = np.subtract(predictions, train_y)
    sum_delta = (alpha / m) * train_x.transpose().dot(errors);
    theta = theta - sum_delta;
    cost_history[i] = compute_cost(train_x, train_y, theta)  
  return theta, cost_history


#updating the parameters of our gradient descent function, learning rate, and how many times function is to be performed
theta = np.zeros(5)
iterations = 10000;
alpha = 0.05;

#calling the gradient descent function
theta, cost_history = gradient_descent(train_x, train_y, theta, alpha, iterations)

#feeding in testing data, printing first five values of predicted outcomes in dataframe format, #printing RMSE and R^2 values
normalize_test_data = ((np.array(test_x) - mu) / sigma)
normalize_test_data = np.hstack((np.ones((n,1)), normalize_test_data))
gas_val_pred = normalize_test_data.dot(theta)
gas_val_df=pd.DataFrame(gas_val_pred)

#calculating and printing RMSE
y_actual = test_y
y_predicted = gas_val_pred 
MSE = np.square(np.subtract(y_actual,y_predicted)).mean()  
RMSE = math.sqrt(MSE)
print("Root Mean Square Error: %s \n" %(RMSE))

#Calculating and Printing R^2 
actual = test_y
predict = gas_val_pred
corr_matrix = np.corrcoef(actual, predict)
corr = corr_matrix[0,1]
R_sq = corr**2
print("r^2 value: %s \n" %(R_sq))

print("The first five predicted values:")
gas_val_df.head()


Total no of training examples (m) = 336000 
 Total number of test values (n) = 84000 

Root Mean Square Error: 0.13640702763058893 

r^2 value: 0.7069188949847561 

The first five predicted values:


,0
0,53.747559
1,53.998710
2,54.330952
3,54.145090
4,53.598414
